# POSTTEST 7 Praktikum Kercedasan Buatan
---
(*2109106063* - Zahra Salsabila - B121)

# Import Library

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import os

# Menampilkan Sample Data Gambar

In [2]:
class_name = ['Anaphalis_Javanica', 'Leontopodium_Alpinum', 'Leucogenes_Grandiceps']

for i in range(3):
    # mengambil sample data gambar dari folder train
    nama_gambar = os.path.join('Edelweis/Train', class_name[i], class_name[i]+'_0.png')
    gambar = plt.imread(nama_gambar)
    plt.figure(figsize=(10,10))
    plt.subplot(1, 3, 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(gambar)
    plt.xlabel(class_name[i])
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'Edelweis/Train\\Anaphalis_Javanica\\Anaphalis_Javanica_0.png'

# Preprocessing Data Gambar (Augmentasi Data)

In [ ]:
img_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.1,
    rotation_range=5,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True, 
    vertical_flip=True,
    fill_mode='nearest'
)

# Split Data Menjadi train,val dan test

- File directory

In [ ]:
train_dir = 'Train'
test_dir = 'Test'

- Split dataset ke dalam train

In [ ]:
train_gen = img_datagen.flow_from_directory(
    train_dir,
    subset="training",
    seed=42,
    target_size=(128, 128),
    batch_size=16,
    shuffle = True,
    class_mode="categorical",
)

- Split dataset ke dalam validation

In [ ]:
valid_gen = img_datagen.flow_from_directory(
    train_dir,
    subset="validation",
    seed=42,
    target_size=(128, 128),
    batch_size=16,
    shuffle = True,
    class_mode="categorical",
)

- Split dataset ke dalam test 

In [ ]:
test_gen = img_datagen.flow_from_directory(
    test_dir,
    seed=42,
    target_size=(128, 128),
    batch_size=16,
    shuffle = True,
    class_mode="categorical",
)

# Membuat Model Sequential

- Model

In [ ]:
model = tf.keras.Sequential([
    InputLayer(input_shape=(128, 128, 3)),
    Conv2D(16, (3,3), activation='relu' ),
    MaxPooling2D(2, 2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(2048, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

- Summary

In [ ]:
model.summary()

# Compiling model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Fit Model
- Fungsi callback

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=0, 
    mode='auto')

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,
    patience=5)

callbacks = [early_stopping_callback, lr_reducer]

- Fit model

In [ ]:
model_fit = model.fit(
    train_gen,
    epochs = 10,
    validation_data = valid_gen,
    callbacks = callbacks,
    verbose = 1)

# Evaluasi Model

In [ ]:
loss, accuracy = model.evaluate(test_gen)
print("-------------------------------------------------")
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')
print("-------------------------------------------------")

# Grafik Pergerakan Metrics

In [ ]:
acc = model_fit.history['accuracy']
val_acc = model_fit.history['val_accuracy']
loss_ = model_fit.history['loss']
val_loss_ = model_fit.history['val_loss']

In [ ]:
plt.subplot(1,2,1)
plt.plot(acc)
plt.plot(val_acc)
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='upper left')
plt.show()

plt.subplot(1,2,2)
plt.plot(loss)
plt.plot(val_loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss', 'validation Loss'], loc='upper right')
plt.show()